# Introduction
This section aims to provide simple statistics for the TyDi data set as a starter for the project. A clear explanation of results can be found in the report.

In [1]:
import pandas as pd
import torch
import string
import pandas as pd
import numpy as np
import torch
from tabulate import tabulate
import matplotlib.pyplot as plt
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using GPU")
else:
  print("Using CPU")

Using CPU


# Prepare data
The script below creates the dictionaries test_set_dict and val_set_dict for each language given as input. It is used in every section in the project.

In [4]:
%run /app/prepare_data.py english

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


README.md:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

(…)-00000-of-00001-af2f3eaa87d1aa8b.parquet:   0%|          | 0.00/71.6M [00:00<?, ?B/s]

(…)-00000-of-00001-1f04eb244a33fa1b.parquet:   0%|          | 0.00/7.49M [00:00<?, ?B/s]

...

Adding  english to dict


Filter:   0%|          | 0/116067 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

Map:   0%|          | 0/7389 [00:00<?, ? examples/s]

Map:   0%|          | 0/990 [00:00<?, ? examples/s]

In [171]:
#showcasing the format of the data sets
train_set_dict, val_set_dict, languages

({'english': Dataset({
      features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'answerable'],
      num_rows: 7389
  })},
 {'english': Dataset({
      features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'answerable'],
      num_rows: 990
  })},
 ['english'])

In [172]:
english_example = train_set.filter(lambda example: example['language'] == 'english')
print(english_example[:]['question_text'][0]) # The question text
print(english_example[:]['document_plaintext'][0]) # The document, which may have the answer to the question
print(english_example[:]['annotations'][0]) # The answer to the document - empty if the answer is not within the document text.
print('\n')
print(english_example[:]['question_text'][-1])
print(english_example[:]['document_plaintext'][-1])
print(english_example[:]['annotations'][-1]) # this one will be empty: the answer is not in the document.

When was quantum field theory developed?
Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1
{'answer_start': [159], 'answer_text': ['1920s']}


What is the population of Mahwah, NJ?
The previous mayor, Bill Laforet faced a recall election in November 2018, after a resident group submitted in June a list of 5,000 petition signatures that they had collected calling for the action, in excess of the 25% needed to place the measure in front of voters.[85] In the November 2018 general election, Laforet was recalled from office and John Roth was elected mayor. The successful recall was the first in the county for at least 25 years.[86]
{'answer_start': [-1], 'answer_text': ['']}


# Section 1

### Basic Statistics

In [174]:
table_data = []
def unique_words(data):
  word_to_ix = {}
  for sent in data:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
  return word_to_ix
for language in languages:
    train_set = train_set_dict[language]
    val_set = val_set_dict[language]

    train_questions = train_set['question_words']
    val_questions = val_set['question_words']

    stop_words = set(stopwords.words(language.lower()))

    train_word_counts = len(train_questions)
    val_word_counts = len(val_questions)
    train_answerable_ratio = np.mean([len(x['annotations']['answer_text'][0]) > 0 for x in train_set])
    val_answerable_ratio = np.mean([len(nltk.word_tokenize(x['annotations']['answer_text'][0])) > 0 for x in val_set])

    flat_train_q = [item for sublist in train_questions for item in sublist]
    flat_val_q = [item for sublist in val_questions for item in sublist]

    # Calculate the total length of inner lists
    train_avg_words_per_question = sum(len(inner_list) for inner_list in train_questions) / len(train_questions)
    val_avg_words_per_question = sum(len(inner_list) for inner_list in val_questions) / len(val_questions)

    train_total_words = train_word_counts * train_avg_words_per_question
    val_total_words = val_word_counts * val_avg_words_per_question

    train_unique_words = len(unique_words(train_questions))
    val_unique_words = len(unique_words(val_questions))

    table_data.append([
        language,
        "Train Set",
        len(train_questions),
        np.round(train_avg_words_per_question, 2),
        np.round(train_answerable_ratio, 2),
        np.round(train_total_words, 2),
        np.round(train_unique_words, 2)
    ])
    table_data.append([
        "",
        "Validation Set",
        len(val_questions),
        np.round(val_avg_words_per_question, 2),
        np.round(val_answerable_ratio, 2),
        np.round(val_total_words, 2),
        np.round(val_unique_words, 2)
    ])

headers = ["Language", "Dataset", "Number of questions", "Avg. Words per question", "Answerable ratio", "Avg. no. of words in questions", "No. of unique words"]

table = tabulate(table_data, headers, tablefmt="latex_raw")
print(table)

\begin{tabular}{llrrrrr}
\hline
 Language   & Dataset        &   Number of questions &   Avg. Words per question &   Answerable ratio &   Avg. no. of words in questions &   No. of unique words \\
\hline
 english    & Train Set      &                  7389 &                      8.06 &                0.5 &                            59533 &                  5018 \\
            & Validation Set &                   990 &                      8.2  &                0.5 &                             8122 &                  1230 \\
\hline
\end{tabular}


### Most frequent words by language in the data set

In [178]:
combined_data = {'Language': [], 'Type': [], 'Top Word': [], 'Count': []}
for country in languages:
    stop_words = set(stopwords.words(country.lower()))

    # we decided to remove additional characters that were top words
    # for the countries because they are not particulary interesting
    additional_characters = list(string.punctuation) + ["``", "''", "؟", ",", "'s"]

    doc_words = [word for sublist in train_set_dict[country]['doc_text_words'] for word in sublist if word not in stop_words and word not in additional_characters]
    question_words = [word for sublist in train_set_dict[country]['question_words'] for word in sublist if word not in stop_words and word not in additional_characters]

    top_words_doc = pd.Series(doc_words).value_counts()[:5]
    top_words_q = pd.Series(question_words).value_counts()[:5]


    # we translate to english to understand which words are used the most
    
    if language == 'english': # If we are analyzing the english data set, there is no reason to translate
        translated_words_doc = top_words_doc.index
        translated_words_q = top_words_q.index
    else:
        translator = Translator()

        translated_words_doc = [translator.translate(word, src=country, dest='en').text for word in top_words_doc.index]
        translated_words_q = [translator.translate(word, src=country, dest='en').text for word in top_words_q.index]

    for word, count in zip(translated_words_doc, top_words_doc.values):
        combined_data['Language'].append(country.capitalize())
        combined_data['Type'].append('doc')
        combined_data['Top Word'].append(word)
        combined_data['Count'].append(count)

    for word, count in zip(translated_words_q, top_words_q.values):
        combined_data['Language'].append(country.capitalize())
        combined_data['Type'].append('question')
        combined_data['Top Word'].append(word)
        combined_data['Count'].append(count)

combined_df = pd.DataFrame(combined_data)
print(combined_df)

  Language      Type Top Word  Count
0  English       doc      The   7661
1  English       doc       In   2162
2  English       doc    first   1875
3  English       doc        1   1802
4  English       doc        2   1521
5  English  question     When   2242
6  English  question     What   2103
7  English  question      How   1300
8  English  question      Who   1070
9  English  question    first    948


### A initial model: A rule-based classifier
The model is supervised and predicts if a document contains an answer to a question. If the document and question share 2 or more words, it predicts "yes"; otherwise, "no."

In [179]:
def oracle(data, language):
    """Generates labels based on whether answers are present in annotations."""
    return [0 if elem['annotations']['answer_text'][0] == '' else 1 for elem in data[language]]

def has_sufficient_common_words(question_words, doc_words, n):
    """Checks if a question is answerable by counting common words with the document."""
    return len(set(question_words) & set(doc_words)) >= n

def rule_based_classifier(data, country, n):
    """
    Classifies questions as answerable or not based on common words between question and document.
    
    Args:
        data (dict): Dataset containing question and document words.
        country (str): Language identifier 
        n (int): Minimum number of common words required to classify as answerable.
    
    Returns:
        list: Binary classifications for each question.
    """
    country_data = data[country]
    question_words = country_data['question_words']
    doc_words = country_data['doc_text_words']

    return [has_sufficient_common_words(q, doc_words[i], n) for i, q in enumerate(question_words)]

def compute_accuracies(data, languages, n):
    accuracies = {}
    for lang in languages:
        predictions = rule_based_classifier(data, lang, n)
        labels = oracle(data, lang)
        accuracy = np.mean(np.array(predictions) == np.array(labels))
        accuracies[lang] = np.round(accuracy, 2)
    return accuracies

accuracies = compute_accuracies(val_set_dict, languages, 2)

# Print accuracies for each language
for lang, acc in accuracies.items():
    print(f"Accuracy for the {lang} language: {acc}")



Accuracy for the english language: 0.62
